<a href="https://colab.research.google.com/github/enderunknown222/MA/blob/Ver0.1/MA4550_Project_ver0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import numpy as np
import pandas as pd
import io
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None

In [7]:
from google.colab import files
uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['data.csv']))

Saving data.csv to data.csv


In [200]:
#test data size
x=190

Part 1. Predict without any stats

In [ ]:
#Change name to ID
results = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results[j][i]==Name[k]:
        results[j][i]=k
results

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.0,3.0,1.8,0
1,5,0.7,1.0,0.0,0.8,15
2,10,3.3,4.0,3.0,0.6,8
3,18,1.1,0.0,2.0,1.5,13
4,17,0.5,0.0,3.0,2.2,9
...,...,...,...,...,...,...
375,0,NaN,NaN,NaN,NaN,2
376,18,NaN,NaN,NaN,NaN,15
377,14,NaN,NaN,NaN,NaN,3
378,10,NaN,NaN,NaN,NaN,5


In [ ]:
traindata=results.iloc[0:x,:]
preddata=results.iloc[x:380,:]
newresults=results
x_train = traindata[['Home', 'Away']]
y_train = traindata[['HScore', 'AScore']]
x_pred = preddata[['Home', 'Away']]
newresults

,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.0,3.0,1.8,0
1,5,0.7,1.0,0.0,0.8,15
2,10,3.3,4.0,3.0,0.6,8
3,18,1.1,0.0,2.0,1.5,13
4,17,0.5,0.0,3.0,2.2,9
...,...,...,...,...,...,...
375,0,NaN,NaN,NaN,NaN,2
376,18,NaN,NaN,NaN,NaN,15
377,14,NaN,NaN,NaN,NaN,3
378,10,NaN,NaN,NaN,NaN,5


In [ ]:
model = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train, y_train)

In [ ]:
goals = model.predict(x_pred)

In [ ]:
for i in range(380-x):
  for j in range(1):
    if j==0:
      k='HScore'
    if j==1:
      k='AScore'
    newresults[k][x+i]=goals[i][j]
newresults

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Home,HxG,HScore,AScore,AxG,Away
0,7,0.2,0.000000,3.000000,1.8,0
1,5,0.7,1.000000,0.000000,0.8,15
2,10,3.3,4.000000,3.000000,0.6,8
3,18,1.1,0.000000,2.000000,1.5,13
4,17,0.5,0.000000,3.000000,2.2,9
...,...,...,...,...,...,...
375,0,NaN,0.333333,2.000000,NaN,2
376,18,NaN,0.707107,0.707107,NaN,15
377,14,NaN,1.242641,1.878680,NaN,3
378,10,NaN,2.242641,0.585786,NaN,5


In [67]:
leaderboard.reset_index()
lb = {'Team': Name,
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
leaderboard = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts'])
for i in range(380):
  if newresults['HScore'][i]>newresults['AScore'][i]:
    leaderboard['Win'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=3
    leaderboard['Loss'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=0
  if newresults['HScore'][i]==newresults['AScore'][i]:
    leaderboard['Draw'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=1
    leaderboard['Draw'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=1
  if newresults['HScore'][i]<newresults['AScore'][i]:
    leaderboard['Loss'][newresults['Home'][i]]+=1
    leaderboard['Pts'][newresults['Home'][i]]+=0
    leaderboard['Win'][newresults['Away'][i]]+=1
    leaderboard['Pts'][newresults['Away'][i]]+=3

NameError: ignored

In [ ]:
leaderboard.sort_values(by=['Pts','Win']，inplace=True, ascending=False)
leaderboard

SyntaxError: ignored

Part2: Using xG to predict the goal efficiency of each team in each match

In [165]:
#Change name to ID
results2 = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results2[j][i]==Name[k]:
        results2[j][i]=k
results2.insert(1,'HGoalEff',np.zeros(380), True)
results2.insert(6,'AGoalEff',np.zeros(380), True)
results2

,Home,HGoalEff,HxG,HScore,AScore,AxG,AGoalEff,Away
0,7,0.0,0.2,0.0,3.0,1.8,0.0,0
1,5,0.0,0.7,1.0,0.0,0.8,0.0,15
2,10,0.0,3.3,4.0,3.0,0.6,0.0,8
3,18,0.0,1.1,0.0,2.0,1.5,0.0,13
4,17,0.0,0.5,0.0,3.0,2.2,0.0,9
...,...,...,...,...,...,...,...,...
375,0,0.0,NaN,NaN,NaN,NaN,0.0,2
376,18,0.0,NaN,NaN,NaN,NaN,0.0,15
377,14,0.0,NaN,NaN,NaN,NaN,0.0,3
378,10,0.0,NaN,NaN,NaN,NaN,0.0,5


In [166]:
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  GoalEff[2][results2['Home'][i]]+=1
  if results2['AScore'][i]==0:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  GoalEff[2][results2['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-GoalEff[3][results2['Home'][i]])**2
  else:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-GoalEff[3][results2['Home'][i]])**2
  if results2['AScore'][i]==0:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/0.5-GoalEff[3][results2['Away'][i]])**2
  else:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]-GoalEff[3][results2['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]


In [167]:
for i in range(x):
  if results2['HScore'][i]==0:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  if results2['AScore'][i]==0:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
for i in range(380-x):
  results2['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Home'][x+i]],GoalEff[4][results2['Home'][x+i]])
  results2['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Away'][x+i]],GoalEff[4][results2['Away'][x+i]])
results2

,Home,HGoalEff,HxG,HScore,AScore,AxG,AGoalEff,Away
0,7,-0.400000,0.2,0.0,3.0,1.8,0.400000,0
1,5,0.300000,0.7,1.0,0.0,0.8,-1.600000,15
2,10,0.175000,3.3,4.0,3.0,0.6,0.800000,8
3,18,-2.200000,1.1,0.0,2.0,1.5,0.250000,13
4,17,-1.000000,0.5,0.0,3.0,2.2,0.266667,9
...,...,...,...,...,...,...,...,...
375,0,1.535440,NaN,NaN,NaN,NaN,-0.448572,2
376,18,1.740273,NaN,NaN,NaN,NaN,0.144380,15
377,14,-2.644727,NaN,NaN,NaN,NaN,-0.894254,3
378,10,-0.029908,NaN,NaN,NaN,NaN,-0.102603,5


In [168]:
traindata2=results2.iloc[0:x,:]
preddata2=results2.iloc[x:380,:]
newresults2=results2
x_train2 = traindata2[['Home','HGoalEff','AGoalEff','Away']]
y_train2 = traindata2[['HScore', 'AScore']]
x_pred2 = preddata2[['Home','HGoalEff','AGoalEff','Away']]

In [169]:
model2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train2, y_train2)

In [170]:
goals2 = model2.predict(x_pred2)

In [171]:
for i in range(380-x):
  for j in range(2):
    if j==0:
      k='HScore'
    if j==1:
      k='AScore'
    newresults2[k][x+i]=goals2[i][j]
newresults2

,Home,HGoalEff,HxG,HScore,AScore,AxG,AGoalEff,Away
0,7,-0.400000,0.2,0.000000,3.000000,1.8,0.400000,0
1,5,0.300000,0.7,1.000000,0.000000,0.8,-1.600000,15
2,10,0.175000,3.3,4.000000,3.000000,0.6,0.800000,8
3,18,-2.200000,1.1,0.000000,2.000000,1.5,0.250000,13
4,17,-1.000000,0.5,0.000000,3.000000,2.2,0.266667,9
...,...,...,...,...,...,...,...,...
375,0,1.535440,NaN,0.753835,2.548828,NaN,-0.448572,2
376,18,1.740273,NaN,1.286031,2.144124,NaN,0.144380,15
377,14,-2.644727,NaN,0.000000,1.688378,NaN,-0.894254,3
378,10,-0.029908,NaN,1.319282,1.715031,NaN,-0.102603,5


In [172]:
lb = {'Team': Name,
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
leaderboard2 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts'])
for i in range(380):
  if newresults2['HScore'][i]>newresults2['AScore'][i]:
    leaderboard2['Win'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=3
    leaderboard2['Loss'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=0
  if newresults2['HScore'][i]==newresults2['AScore'][i]:
    leaderboard2['Draw'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=1
    leaderboard2['Draw'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=1
  if newresults2['HScore'][i]<newresults2['AScore'][i]:
    leaderboard2['Loss'][newresults2['Home'][i]]+=1
    leaderboard2['Pts'][newresults2['Home'][i]]+=0
    leaderboard2['Win'][newresults2['Away'][i]]+=1
    leaderboard2['Pts'][newresults2['Away'][i]]+=3

In [175]:
leaderboard2.sort_values(by=['Pts','Win'],inplace=True, ascending=False)
leaderboard2

,Team,Win,Loss,Draw,Pts
10,Liverpool,27.0,8.0,3.0,84.0
16,Tottenham,25.0,10.0,3.0,78.0
15,Southampton,23.0,13.0,2.0,71.0
19,Wolves,23.0,13.0,2.0,71.0
6,Everton,20.0,16.0,2.0,62.0
18,West Ham,20.0,16.0,2.0,62.0
11,Manchester City,19.0,16.0,3.0,60.0
0,Arsenal,19.0,18.0,1.0,58.0
5,Crystal Palace,19.0,18.0,1.0,58.0
1,Aston Villa,19.0,19.0,0.0,57.0


Repeat Part 2 500 times to find the expected win percentage

In [202]:
y=500
results2 = data.iloc[:,1:7]
Name=['Arsenal','Aston Villa','Brighton','Burnley','Chelsea','Crystal Palace','Everton','Fulham','Leeds United','Leicester City','Liverpool','Manchester City','Manchester Utd','Newcastle Utd','Sheffield Utd','Southampton','Tottenham','West Brom','West Ham','Wolves']
for i in range(380):
  for j in ['Home','Away']:
    for k in range(20):
      if results2[j][i]==Name[k]:
        results2[j][i]=k
results2.insert(1,'HGoalEff',np.zeros(380), True)
results2.insert(6,'AGoalEff',np.zeros(380), True)
GoalEff=[np.arange(20),np.zeros(20),np.zeros(20),np.zeros(20),np.zeros(20)]  
aclb = {'Team': Name,
  'Champ':np.zeros((20)),
  'Win%':np.zeros((20))
    }  
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    GoalEff[1][results2['Home'][i]]+=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  GoalEff[2][results2['Home'][i]]+=1
  if results2['AScore'][i]==0:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    GoalEff[1][results2['Away'][i]]+=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
  GoalEff[2][results2['Away'][i]]+=1
for i in range(20):
  GoalEff[3][i]=GoalEff[1][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/0.5-GoalEff[3][results2['Home'][i]])**2
  else:
    GoalEff[4][results2['Home'][i]]+=((results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]-GoalEff[3][results2['Home'][i]])**2
  if results2['AScore'][i]==0:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/0.5-GoalEff[3][results2['Away'][i]])**2
  else:
    GoalEff[4][results2['Away'][i]]+=((results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]-GoalEff[3][results2['Away'][i]])**2
for i in range(20):
  GoalEff[4][i]=GoalEff[4][i]/GoalEff[2][i]
for i in range(x):
  if results2['HScore'][i]==0:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/0.5
  else:
    results2['HGoalEff'][i]=(results2['HScore'][i]-results2['HxG'][i])/results2['HScore'][i]
  if results2['AScore'][i]==0:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/0.5
  else:
    results2['AGoalEff'][i]=(results2['AScore'][i]-results2['AxG'][i])/results2['AScore'][i]
for a in range(y):
    for i in range(380-x):
      results2['HGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Home'][x+i]],GoalEff[4][results2['Home'][x+i]])
      results2['AGoalEff'][x+i]=np.random.normal(GoalEff[3][results2['Away'][x+i]],GoalEff[4][results2['Away'][x+i]])
    traindata2=results2.iloc[0:x,:]
    preddata2=results2.iloc[x:380,:]
    newresults2=results2
    x_train2 = traindata2[['Home','HGoalEff','AGoalEff','Away']]
    y_train2 = traindata2[['HScore', 'AScore']]
    x_pred2 = preddata2[['Home','HGoalEff','AGoalEff','Away']]
    model2 = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(x_train2, y_train2)
    goals2 = model2.predict(x_pred2)
    for i in range(380-x):
      for j in range(2):
        if j==0:
          k='HScore'
        if j==1:
          k='AScore'
        newresults2[k][x+i]=goals2[i][j]
    lb = {'Team': np.arange(20),
    'Win':np.zeros((20)),
    'Loss':np.zeros((20)),
    'Draw':np.zeros((20)),
    'Pts':np.zeros((20))
      }
    leaderboard2 = pd.DataFrame(lb, columns= ['Team','Win','Loss','Draw','Pts']) 
    for i in range(380):
      if newresults2['HScore'][i]>newresults2['AScore'][i]:
        leaderboard2['Win'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=3
        leaderboard2['Loss'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=0
      if newresults2['HScore'][i]==newresults2['AScore'][i]:
        leaderboard2['Draw'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=1
        leaderboard2['Draw'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=1
      if newresults2['HScore'][i]<newresults2['AScore'][i]:
        leaderboard2['Loss'][newresults2['Home'][i]]+=1
        leaderboard2['Pts'][newresults2['Home'][i]]+=0
        leaderboard2['Win'][newresults2['Away'][i]]+=1
        leaderboard2['Pts'][newresults2['Away'][i]]+=3
    c=leaderboard2.loc[leaderboard2['Pts'].idxmax(), 'Team']
    aclb['Champ'][c]+=1
accleaderboard = pd.DataFrame(aclb, columns= ['Team','Champ','Win%'])
accleaderboard['Win%']=accleaderboard['Champ']/y*100
accleaderboard.sort_values(by=['Champ'],inplace=True, ascending=False)
accleaderboard

,Team,Champ,Win%
11,Manchester City,306.0,61.2
12,Manchester Utd,88.0,17.6
10,Liverpool,69.0,13.8
9,Leicester City,27.0,5.4
6,Everton,6.0,1.2
16,Tottenham,2.0,0.4
1,Aston Villa,2.0,0.4
18,West Ham,0.0,0.0
17,West Brom,0.0,0.0
15,Southampton,0.0,0.0


In [198]:
accleaderboard['Win%']

11    0.240
9     0.225
12    0.220
6     0.150
1     0.055
10    0.035
4     0.025
0     0.015
15    0.010
16    0.010
18    0.010
3     0.005
8     0.000
7     0.000
5     0.000
2     0.000
13    0.000
14    0.000
17    0.000
19    0.000
Name: Win%, dtype: float64